In [ ]:
import os

import utils

import seabeepy as sb
from seabeepy.config import SETTINGS

In [ ]:
# Login to MinIO
client = sb.storage.minio_login(
    user=SETTINGS.MINIO_ACCESS_ID, password=SETTINGS.MINIO_SECRET_KEY
)

# Publish ODM missions

This notebook publishes the "best" ODM missions to GeoNode for visual comparison to the Pix4D reference dataset.

## 1. User input

In [ ]:
# Version of parameter combo file to use
param_version = 0

# Run IDs within 'param_version' to publish
run_ids = [5, 7, 12]

## 2. Update config. files for publishing

The code below updates the config. files so they will be picked up for publishing on the GeoNode next time the pipeline runs (every hour).

In [ ]:
res_dir = f"/home/notebook/shared-seabee-ns9879k/niva-tidy/2025/odm_tuning/params_v{param_version:02d}"
for run_id in run_ids:
    mission_dir = os.path.join(res_dir, f"run-{param_version:02d}-{run_id:03d}")

    # Update config
    data = sb.ortho.parse_config(mission_dir)
    data["publish"] = True
    utils.update_config(mission_dir, data, client, overwrite=True)

    # Delete standradised orthophoto (so it gets processed again)
    ortho_name = sb.ortho.get_layer_name(mission_dir) + ".tif"
    ortho_path = os.path.join(mission_dir, "orthophoto", ortho_name)
    sb.storage.delete_file(ortho_path, client)